In [1]:
import board
import random
import numpy as np
import time
 
# Randomly create 8 initial board states
def get_init_encodings(board_size, n_queens):
  encodings = []
 
  for i in range(board_size):
    encode = ""
    for j in range(n_queens):
      queen_loc = random.randint(0, n_queens - 1)
      encode += str(queen_loc)
    encodings.append(encode)
 
  return encodings
 
 
# Given an encoding, instantiate a CustomBoard object
def decode(encoding):
  cb = board.CustomBoard(encoding)
  cb.fitness()
 
  return cb
 
 
# Given a list of encodings, decode them into a list of CustomBoard objects while
# simultaniously taking the summation of each object's fitness
def create_states(encodings):
  states = []
  total_fit = 0
 
  for i in range(len(encodings)):
    cb = decode(encodings[i])
    total_fit += cb.get_fit()
    states.append(cb)
 
  return [states, total_fit]
 
 
# Normalize a list of CustomBoard's fitnesses
def normalize(total_fit, states):
  normalized = []
 
  for b in states:
    # Calculate total none attacking pairs of all states.
    # Board fitness is defined as number of attacking pairs so subtract that by 
    # 10 and take abs to get none attacking pairs.
    norm = abs(b.fit - 10) / abs(total_fit - 80)
    normalized.append(norm)
  
  return normalized
 
 
# Randomly select a state by turning norm into a probability vector and choosing
# from it.
def selection(norm, board_size):
  np_norm = np.array(norm)
  cum_sum = np.cumsum(np_norm)
  prob = np.append(0, cum_sum)
  rand_val = np.random.rand()
 
  for i in range(board_size):
    if prob[i] < rand_val <= prob[i + 1]:
      return i
 
  return None
 
 
# Given a list of encodings and two locations within the size of that list,
# randomly perform crossover.
def crossover(encodings, enc_size, i, j):
  e1 = encodings[i]
  e2 = encodings[j]
  rand_val = random.randint(0, enc_size - 1)
 
  # Perform crossover between two encodings by going to a random index and
  # swapping everything after it with the encoding next to it and vice versa.
  temp = e1
  mut1 = e1[:rand_val] + e2[rand_val: enc_size + 1]
  mut2 = e2[:rand_val] + temp[rand_val: enc_size + 1]
 
  # Replace the previous encodings with the crossovers
  encodings[i] = mut1
  encodings[j] = mut2
 
  return encodings
 
 
# Given an encoding, potentially mutate a character in it.
def mutate(encoding, enc_size):
  mut_loc = random.randint(0, enc_size)
 
  # If mut_loc is 0, don't mutate, otherwise go to mut_loc and pick a random
  # value to put into it.
  if mut_loc != 0:
    mut_val = random.randint(0, enc_size - 1)
    return encoding[:mut_loc - 1] + str(mut_val) + encoding[mut_loc:]
 
  # Mutation didn't occur
  return encoding
 
 
def main():
  TOTAL_STATES = 8
  N_QUEENS = 5
  RESTART_LIMIT = 100
  flag = False
  restarts = -1
  start = time.clock()
 
  # Keep trying to find a solution until flag is true
  while not flag:
    counter = 0
    restarts += 1
    # Generate 8 initial board encodings and states where an encoding is the pos
    # of a queen on a row
    encodings = get_init_encodings(TOTAL_STATES, N_QUEENS)
    s = create_states(encodings)
 
    # If counter reaches the limit, the board might be stuck in a local minima so
    # restart
    while counter <= RESTART_LIMIT and not flag:
      states = s[0]
      total_fit = s[1]
 
      # Normalize the fitness of states
      normalized = normalize(total_fit, states)
 
      # Choose new encodings based on the results of normalization
      new_encodings = []
      for i in range(TOTAL_STATES):
        select = selection(normalized, TOTAL_STATES)
        new_encodings.append(encodings[select])
 
      # Perform crossover on two encodings at a time for all encodings
      for i in range(0, TOTAL_STATES, 2):
        new_encodings = crossover(encodings, N_QUEENS, i, i + 1)
 
      # Perfrom mutation on the encodings
      for i in range(TOTAL_STATES):
        mutate(new_encodings[i], N_QUEENS)
 
      # Create new states with the new encodings
      encodings = new_encodings
      s = create_states(encodings)
 
      # Check the new states for a solution
      for i in s[0]:
        if i.fit == 0:
          print("SOLUTION")
          i.show()
          print("Restarts:", restarts)
          flag = True
          break
      # A solution was not found so try again
      counter += 1
 
  # Runtime
  print("Running time: {0:.0f}".format((time.clock() - start) * 1000), "ms")
 
main()

SOLUTION
[[0 0 0 0 1]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [1 0 0 0 0]
 [0 0 1 0 0]]
Fitness:  0
Restarts: 1
Running time: 61 ms


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:111: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:161: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
